# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala Gradient Ascent

In [ ]:
import smile.plot._
import scala.io.Source
import org.clustering4ever.math.distances.scalar.Euclidean
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.scala.clustering.meanshift.{GradientAscent, GradientAscentArgs}
import org.clustering4ever.enums.KernelNature
import scala.collection.{immutable, mutable}
import org.clustering4ever.kernels._
import org.clustering4ever.shapeless.VMapping

import smile.plot._
import scala.io.Source
import org.clustering4ever.math.distances.scalar.Euclidean
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.scala.clustering.meanshift.{GradientAscent, GradientAscentArgs}
import org.clustering4ever.enums.KernelNature
import scala.collection.{immutable, mutable}
import org.clustering4ever.kernels._
import org.clustering4ever.shapeless.VMapping


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-01-17 23:56:38--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.1’

     0K .......                                               100%  101M=0s

2019-01-17 23:56:38 (101 MB/s) - ‘/tmp/aggregation.csv.1’ saved [8063/8063]

:sh: Scheme missing.
--2019-01-17 23:56:38--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-01-17 23:56:38--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.1’

     0K .                                                     100%

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val path = "/tmp/aggregation.csv"
val data = Source.fromFile(path).getLines.toSeq.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*)).toParArray.zipWithIndex
    .map{ case (v, id) => EasyClusterizable(id, ScalarVector(v)) }
val labelsPath = "/tmp/labels"

path: String = /tmp/aggregation.csv
data: scala.collection.parallel.mutable.ParArray[org.clustering4ever.clusterizables.EasyClusterizable[Int,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = ParArray(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@ffbe1d,Vector()), EasyClusterizable(1,Vectorizable(ScalarVector(ArrayBuffer(14.9, 27.55))),ScalarVector(ArrayBuffer(14.9, 27.55)),shapeless.HMap@1335e7ec,Vector()), EasyClusterizable(2,Vectorizable(ScalarVector(ArrayBuffer(14.45, 28.35))),ScalarVector(ArrayBuffer(14.45, 28.35)),shapeless.HMap@75e8cbc9,Vector()), EasyClusterizable(3,Vecto...

## Parameters 

In [ ]:
val classicEuclidean = new Euclidean[mutable.ArrayBuffer[Double]](false)
val k = 40
val epsilon = 0.00001
val maxNumberIter = 50

classicEuclidean: org.clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without squared root applied
k: Int = 40
epsilon: Double = 1.0E-5
maxNumberIter: Int = 50


In [ ]:
val knnRealKernel = KernelArgsKnnReal(k = 50, metric = classicEuclidean)
val knnEuclideanKernel = KernelArgsEuclideanKnn(k = 100, metric = classicEuclidean)
val gaussianArgs = KernelArgsGaussian(bandwidth = 0.001, metric = classicEuclidean)
val flatArgs = KernelArgsFlat(bandwidth = 0.75, metric = classicEuclidean, lambda = 1D)
val sigmoidArgs = KernelArgsSigmoid(a = 1D, b = 3D)

knnRealKernel: org.clustering4ever.kernels.KernelArgsKnnReal[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsKnnReal(50,Euclidean distance without squared root applied)
knnEuclideanKernel: org.clustering4ever.kernels.KernelArgsEuclideanKnn[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsEuclideanKnn(100,Euclidean distance without squared root applied)
gaussianArgs: org.clustering4ever.kernels.KernelArgsGaussian[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsGaussian(0.001,Euclidean distance without squared root applied)
flatArgs: org.clustering4ever.kernels.KernelArgsFlat[scala.collection.mutable.ArrayB...

In [ ]:
val gaussian = KernelGaussian(gaussianArgs)
val flat = KernelFlat(flatArgs)
val sigmoid = KernelSigmoid[mutable.ArrayBuffer[Double]](sigmoidArgs)
val knnReal = KnnKernelReal(knnRealKernel)
val knnEuc = KnnKernelEuclidean(knnEuclideanKernel)

gaussian: org.clustering4ever.kernels.KernelGaussian[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KernelGaussian(KernelArgsGaussian(0.001,Euclidean distance without squared root applied))
flat: org.clustering4ever.kernels.KernelFlat[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KernelFlat(KernelArgsFlat(0.75,Euclidean distance without squared root applied,1.0))
sigmoid: org.clustering4ever.kernels.KernelSigmoid[scala.collection.mutable.ArrayBuffer[Double]] = KernelSigmoid(KernelArgsSigmoid(1.0,3.0))
knnReal: org.clustering4ever.kernels.KnnKernelReal[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean,org.clustering4ever.kernels.KernelA...

## Run the algorithm

In [ ]:
val idWhereKeepNewVectorization = 9

val t1 = System.currentTimeMillis
val convergedDS =  GradientAscent.run(data, epsilon, maxNumberIter, knnEuc, classicEuclidean, idWhereKeepNewVectorization)
val t2 = System.currentTimeMillis

(t2 - t1) / 1000D

idWhereKeepNewVectorization: Int = 9
t1: Long = 1547766947857
convergedDS: scala.collection.parallel.mutable.ParArray[org.clustering4ever.clusterizables.EasyClusterizable[Int,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = ParArray(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@42657c53,Vector()), EasyClusterizable(1,Vectorizable(ScalarVector(ArrayBuffer(14.9, 27.55))),ScalarVector(ArrayBuffer(14.9, 27.55)),shapeless.HMap@163d8292,Vector()), EasyClusterizable(2,Vectorizable(ScalarVector(ArrayBuffer(14.45, 28.35))),ScalarVector(ArrayBuffer(14.45, 28.35)),shapeless.HMap@1f6c0351,V...

0.588



# Plot original and converged datsasets

In [ ]:
val vMapping = VMapping[Int, ScalarVector[mutable.ArrayBuffer[Double]]]
val res = convergedDS.map(_.vectorized.get(idWhereKeepNewVectorization)(vMapping).get.vector.toArray).toArray
plot(res)

vMapping: org.clustering4ever.shapeless.VMapping[Int,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]] = org.clustering4ever.shapeless.VMapping@65c7e3ad
res: Array[Array[Double]] = Array(Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315, 24.303999999999995), Array(8.8315...

Window(javax.swing.JFrame[frame1,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 2,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])